In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import msmrd2.tools as msmrdtls
import msmrd2.analysis as msmrdals
from msmrd2 import msmrdMarkovModel

In [ ]:
# Define useful parameters
dt = 0.00001
timesteps = 20000000
stride = 2500
maxentries = int(timesteps/stride)
fnamebase = '../data/dimer/simDimer'
nfiles = 10

In [ ]:
# Load full trajectories
trajs = []
for i in range(nfiles):
    traj = msmrdals.loadTrajectory(fnamebase, i)
    trajs.append(traj)

In [ ]:
# Plot relative position
diffpos =[]
file=0
for entry in range(maxentries):
    time = trajs[file][entry][0]*dt
    part1pos = np.array(trajs[file][2*entry][1:4])
    part2pos = np.array(trajs[file][2*entry+1][1:4])
    diffpos.append(np.linalg.norm(part1pos - part2pos))
plt.plot(diffpos)

In [ ]:
# Plot relative orientation
difforient =[]
for entry in range(int(maxentries)):
    time = trajs[file][entry][0]*dt
    part1orient = np.array(trajs[file][2*entry][4:8])
    part2orient = np.array(trajs[file][2*entry+1][4:8])
    relorient = msmrdtls.quaternions.relativeOrientation(part1orient, part2orient)
    phi = msmrdtls.quaternions.quat2angle(relorient)
    difforient.append(np.linalg.norm(phi))
plt.plot(difforient)

In [ ]:
dtrajs = []
for i in range(nfiles):
    dtraj = msmrdals.loadDiscreteTrajectory(fnamebase, i)
    dtrajs.append(dtraj)

In [ ]:
# Create distionaries to keep track of specific events. Needed to calculate rates
boundstates = 2
orientations = 6
timeDict, eventDict = msmrdals.createStatesDictionaries(boundstates, orientations)

In [ ]:
dt = trajs[0][2][0] - trajs[0][0][0]
rateDict, timeDict, eventDict = msmrdals.extractRates(dtrajs, timeDict, eventDict)

In [ ]:
rateDict

In [ ]:
a=msmrdMarkovModel(3,6,-1,rateDict)

In [ ]:
a.getRate('b2->b1')

In [ ]:
msmA = a.getMSM('35->b1')

In [ ]:
msmA.

In [ ]:
sliceIndex = a.find('->')